In [1]:
import cudf  # this should work without any errors

In [2]:
!pip install plotly-express

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 15.5 MB/s eta 0:00:00m eta 0:00:01:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 27.2 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 29.3 MB/s eta 0:00:00m eta 0:00:010:00:01


In [3]:
!wget https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet

--2023-12-13 20:25:11--  https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet
Resolving data.rapids.ai (data.rapids.ai)... 2600:9000:2335:9a00:1a:8fa4:1000:93a1, 2600:9000:2335:ba00:1a:8fa4:1000:93a1, 2600:9000:2335:aa00:1a:8fa4:1000:93a1, ...
Connecting to data.rapids.ai (data.rapids.ai)|2600:9000:2335:9a00:1a:8fa4:1000:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 477161608 (455M) [binary/octet-stream]
Saving to: ‘nyc_parking_violations_2022.parquet’

nyc_parking_violati 100%[===================>] 455.06M  29.5MB/s    in 16s     

2023-12-13 20:25:29 (27.7 MB/s) - ‘nyc_parking_violations_2022.parquet’ saved [477161608/477161608]



In [4]:
import pandas as pd

In [5]:
# read 5 columns data:
df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

# view a random sample of 10 rows:
df.sample(10)

,Registration State,Violation Description,Vehicle Body Type,Issue Date,Summons Number
14272412,NY,21-No Parking (street clean),SUBN,05/27/2022,8810296448
5169487,NY,70A-Reg. Sticker Expired (NYS),FLAT,10/04/2021,8943474751
9512266,NY,PHTO SCHOOL ZN SPEED VIOLATION,SUBN,02/10/2022,4767541402
15059277,NJ,38-Failure to Dsplay Meter Rec,SUBN,06/03/2022,8866224339
3162659,NY,19-No Stand (bus stop),SUBN,09/07/2021,8966758228
1353666,NY,21-No Parking (street clean),SUBN,06/30/2021,8989415111
5822362,PA,BUS LANE VIOLATION,SW,10/30/2021,4020526412
10492509,NY,PHTO SCHOOL ZN SPEED VIOLATION,SUBN,02/23/2022,4769695391
7276711,MS,PHTO SCHOOL ZN SPEED VIOLATION,UV,12/16/2021,4760841556
10518714,TN,PHTO SCHOOL ZN SPEED VIOLATION,<NA>,03/04/2022,4770731644


In [6]:
(df[["Registration State", "Violation Description"]]  # get only these two columns
 .value_counts()  # get the count of offences per state and per type of offence
 .groupby("Registration State")  # group by state
 .head(1)  # get the first row in each group (the type of offence with the largest count)
 .sort_index()  # sort by state name
 .reset_index()
)

,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [7]:
(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
CARY,1
ISUZ,1
IXMR,1


In [8]:
weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

In [9]:
%%time

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 6.92 s, sys: 2.62 s, total: 9.54 s
Wall time: 6.4 s


,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [10]:
%%time

(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

CPU times: user 1.47 s, sys: 80.7 ms, total: 1.55 s
Wall time: 1.55 s


,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
CARY,1
ISUZ,1
IXMR,1


In [11]:
%%time

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 2.94 s, sys: 426 ms, total: 3.36 s
Wall time: 3.37 s


issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

In [12]:
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [3]:
%load_ext cudf.pandas

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [4]:
%%time

import pandas as pd

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 8.57 s, sys: 2.85 s, total: 11.4 s
Wall time: 8.23 s


,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [4]:
%%time

import pandas as pd

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 8.57 s, sys: 2.85 s, total: 11.4 s
Wall time: 8.23 s


,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [5]:
%%time

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 7.29 s, sys: 1.31 s, total: 8.59 s
Wall time: 8.49 s


issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

In [6]:
%%cudf.pandas.profile

small_df = pd.DataFrame({'a': [0, 1, 2], 'b': ["x", "y", "z"]})
small_df = pd.concat([small_df, small_df])

axis = 0
for i in range(0, 2):
    small_df.min(axis=axis)
    axis = 1

counts = small_df.groupby("a").b.count()

/home/soap1196/miniconda3/envs/rapids-23.12/lib/python3.10/site-packages/cudf/core/dataframe.py:5933: UserWarning: Row-wise operations currently only support int, float and bool dtypes. Non numeric columns are ignored.
  warnings.warn(msg)


../../thread/thread_load.cuh(36): warning: cuda.h: [jitify] File not found
../../thread/thread_store.cuh(36): warning: cuda.h: [jitify] File not found


                                                                                                         
                                        Total time elapsed: 6.623 seconds                                
                                      5 GPU function calls in 6.280 seconds                              
                                      1 CPU function calls in 0.012 seconds                              
                                                                                                         
                                                      Stats                                              
                                                                                                         
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function            ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame           │ 1          │ 0.017       │ 0.017       │ 0          │ 0.000       │ 0.000       │
│ concat              │ 1          │ 0.017       │ 0.017       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.min       │ 1          │ 6.235       │ 6.235       │ 1          │ 0.012       │ 0.012       │
│ DataFrame.groupby   │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ SeriesGroupBy.count │ 1          │ 0.011       │ 0.011       │ 0          │ 0.000       │ 0.000       │
└─────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- DataFrame.min

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=777675;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

In [7]:
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [1]:
!pip install "modin[all]"

  Using cached modin-0.25.1-py3-none-any.whl.metadata (17 kB)
  Using cached dask-2023.12.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached distributed-2023.12.0-py3-none-any.whl.metadata (3.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 2.2 MB/s eta 0:00:001.2 MB/s eta 0:00:01
  Using cached unidist-0.4.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached modin_spreadsheet-0.1.2-py2.py3-none-any.whl (1.8 MB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached partd-1.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached toolz-0.12.0-py3-none-any.whl (55 kB)
  Using cached importlib_metadata-7.0.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached locket-1.0.0-py2.py3-none-any.whl (4.4 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached tblib-3.0.0-py3-none-any.whl.metadata (25 kB)
  Using cached zict-3.0.0-py2.py3-none-any.whl (43 kB)
  Usin

In [2]:
import modin.pandas as pd

In [3]:
import modin.pandas as pd

In [4]:
%%time

import pandas as pd

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 6.49 s, sys: 2.54 s, total: 9.03 s
Wall time: 5.87 s


,Registration State,Violation Description,count
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [5]:
%%time

import pandas as pd

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 6.61 s, sys: 2.69 s, total: 9.3 s
Wall time: 6.26 s


,Registration State,Violation Description,count
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [6]:
%%time

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 1.73 s, sys: 356 ms, total: 2.08 s
Wall time: 2.09 s


issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64